# Put-Call Ratio Time-Series prediction using LSTM

- <b>Using Long Short Term Memory (LSTM) to buid a Time-series prediction model to predict the \"Put-Call Ratio\" on 16th August.</b><br>

- <b>With the value of Put-Call Ratio on 16th August and substituting these values in the \"Train_Dataset\" predicting the Stock Price using Artificial Neural Network (ANN).</b><br>,

- <b> The idea behind the model is to predict the Put-Call Ratio value on 16th August and then predict the Stock Price. To predict the Put-Call Ratio value on 16th August, a LSTM model was created with window size i.e. Multiple Parallel Series. This LSTM picked 3 windows and predicted the rest of the windows one by one (4th window and so on). Since in the dataset sheet_name= \"Put-Call_TS\", the Put-Call Ratio value is iven for 6 days i.e. from 10th August to 15th August. So with the help of Multiple Parallel LSTM model, the prediction of  the Put-Call Ratio was made for the day of 16th August.</b><br>,

- <b>After predicting the Put-Call Ratio value on 16th August, these values were substituted in the \"Train_Dataset\" with Put-Call Ratio value of 16th August. After doing this procedure, the model was trained using 5-layererd ANN and it was fit in to the dataset to predict Stock Price of Test dataset on 16th August.</b><br>,

- <b>Output is Stock Price with Stock Index is in the form of excel workbook \"part-02.xlsx\"</b>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as rf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from statsmodels.tsa.statespace.sarimax import SARIMAX

### Function to create window sequence

In [2]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences)-1:
            break
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [3]:
ds = pd.read_excel("Test_dataset.xlsx", sheet_name="Put-Call_TS")
ds.head()

,Stock Index,Put-Call Ratio,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,2020-08-10 00:00:00,2020-08-11 00:00:00,2020-08-12 00:00:00,2020-08-13 00:00:00,2020-08-14 00:00:00,2020-08-15 00:00:00
1,AC3235,0.8,0.66,0.7724,0.983224,0.901211,0.898368
2,AC3236,0.86,0.732,0.85448,1.08664,1.01704,1.02926
3,AC3237,0.93,0.816,0.95024,1.2073,NaN,1.18196
4,AC3238,1.09,1.008,1.16912,1.48309,1.46106,1.531


## Preprocessing of Put-Call_TS data

### Renaming the features and deleting some unwanted items

In [4]:
ds.rename(columns = {
    'Put-Call Ratio': 'August 10',
    'Unnamed: 2': 'August 11',
    'Unnamed: 3': 'August 12',
    'Unnamed: 4': 'August 13',
    'Unnamed: 5': 'August 14',
    'Unnamed: 6': 'August 15',
}, inplace = True)

In [5]:
ds = ds.drop('Stock Index', axis=1)
ds = ds.iloc[1:, :]
ds.head()

,August 10,August 11,August 12,August 13,August 14,August 15
1,0.8,0.66,0.7724,0.983224,0.901211,0.898368
2,0.86,0.732,0.85448,1.08664,1.01704,1.02926
3,0.93,0.816,0.95024,1.2073,NaN,1.18196
4,1.09,1.008,1.16912,1.48309,1.46106,1.531
5,NaN,1.104,1.27856,1.62099,1.6155,1.70552


### Checking for null values and filling them

In [6]:
ds_null = ds.columns[ds.isnull().any()]
ds[ds_null].isnull().sum()

August 10    429
August 11      2
August 12     57
August 13     14
August 14    105
August 15    239
dtype: int64

In [7]:
ds.columns

Index(['August 10', 'August 11', 'August 12', 'August 13', 'August 14',
       'August 15'],
      dtype='object')

In [8]:
ds["August 10"].fillna(method ='ffill', inplace = True)
ds["August 11"].fillna(method ='ffill', inplace = True)
ds["August 12"].fillna(method ='ffill', inplace = True)
ds["August 13"].fillna(method ='ffill', inplace = True)
ds["August 14"].fillna(method ='ffill', inplace = True)
ds["August 15"].fillna(method ='ffill', inplace = True)

### Window sequences

In [9]:
ds2 = ds.iloc[:,:].values
ds2

array([[0.8       , 0.66      , 0.7724    , 0.983224  , 0.90121088,
        0.89836829],
       [0.86      , 0.732     , 0.85448   , 1.0866448 , 1.01704218,
        1.02925766],
       [0.93      , 0.816     , 0.95024   , 1.2073024 , 1.01704218,
        1.18196192],
       ...,
       [0.67      , 0.504     , 0.59456   , 0.7591456 , 0.65024307,
        0.61477467],
       [0.79      , 0.648     , 0.75872   , 0.9659872 , 0.88190566,
        0.8765534 ],
       [0.74      , 0.588     , 0.69032   , 0.8798032 , 0.78537958,
        0.76747893]])

In [10]:
ds2 = ds2.transpose()
ds2

array([[0.8       , 0.86      , 0.93      , ..., 0.67      , 0.79      ,
        0.74      ],
       [0.66      , 0.732     , 0.816     , ..., 0.504     , 0.648     ,
        0.588     ],
       [0.7724    , 0.85448   , 0.95024   , ..., 0.59456   , 0.75872   ,
        0.69032   ],
       [0.983224  , 1.0866448 , 1.2073024 , ..., 0.7591456 , 0.9659872 ,
        0.8798032 ],
       [0.90121088, 1.01704218, 1.01704218, ..., 0.65024307, 0.88190566,
        0.78537958],
       [0.89836829, 1.02925766, 1.18196192, ..., 0.61477467, 0.8765534 ,
        0.76747893]])

In [11]:
ds2.shape

(6, 3331)

In [12]:
n_steps = 3
X, y = split_sequences(ds2, n_steps)
n_features = X.shape[2]
print(X.shape[2])
print(X)

3331
[[[0.8        0.86       0.93       ... 0.67       0.79       0.74      ]
  [0.66       0.732      0.816      ... 0.504      0.648      0.588     ]
  [0.7724     0.85448    0.95024    ... 0.59456    0.75872    0.69032   ]]

 [[0.66       0.732      0.816      ... 0.504      0.648      0.588     ]
  [0.7724     0.85448    0.95024    ... 0.59456    0.75872    0.69032   ]
  [0.983224   1.0866448  1.2073024  ... 0.7591456  0.9659872  0.8798032 ]]

 [[0.7724     0.85448    0.95024    ... 0.59456    0.75872    0.69032   ]
  [0.983224   1.0866448  1.2073024  ... 0.7591456  0.9659872  0.8798032 ]
  [0.90121088 1.01704218 1.01704218 ... 0.65024307 0.88190566 0.78537958]]]


In [13]:
y

array([[0.983224  , 1.0866448 , 1.2073024 , ..., 0.7591456 , 0.9659872 ,
        0.8798032 ],
       [0.90121088, 1.01704218, 1.01704218, ..., 0.65024307, 0.88190566,
        0.78537958],
       [0.89836829, 1.02925766, 1.18196192, ..., 0.61477467, 0.8765534 ,
        0.76747893]])

## Training the time-series data with LSTM model

In [14]:
ts_lstm = Sequential()
ts_lstm.add(LSTM(90, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
ts_lstm.add(LSTM(90, activation='relu'))
ts_lstm.add(Dense(n_features))
ts_lstm.compile(optimizer='adam', loss='mse')
ts_lstm.fit(X, y, epochs=300, verbose=0)

In [15]:
z = ds2[3:6, :]
z

array([[0.983224  , 1.0866448 , 1.2073024 , ..., 0.7591456 , 0.9659872 ,
        0.8798032 ],
       [0.90121088, 1.01704218, 1.01704218, ..., 0.65024307, 0.88190566,
        0.78537958],
       [0.89836829, 1.02925766, 1.18196192, ..., 0.61477467, 0.8765534 ,
        0.76747893]])

In [16]:
z = z.reshape((1, n_steps, n_features))
vals = ts_lstm.predict(z, verbose=0)

## Train Dataset - Preprocessing

### Dropping of unwanted features/attributes

In [17]:
df = pd.read_excel("Train_dataset.xlsx", sheet_name = "Train_Aug10")
df1 = pd.read_excel("Test_dataset.xlsx", sheet_name = "Test_Aug10")
df2 = pd.read_excel("Test_dataset.xlsx", sheet_name = "Test_Aug10")

In [18]:
df.head()

,Stock Index,Index,Industry,VWAP,General Index,NAV,P/E Ratio,Volumes Traded,Inventory Turnover,Covid Impact (Beta),Tracking Error,Dollar Exchange Rate,Put-Call Ratio,P/B Ratio,Stock Price
0,AA1,NYSE,Real Estate,700.29,12765.84,32.38,45.35,1208599.0,5.05,-0.43,0.0520,1.0,0.61,9.20,156.987852
1,AA2,NYSE,Information Tech,214.52,12765.84,33.25,170.56,13753356.0,3.10,0.23,0.0164,1.0,NaN,6.91,560.859622
2,AA3,NYSE,Information Tech,943.42,12765.84,88.27,143.58,6892003.0,7.70,0.23,0.0396,1.0,NaN,6.09,1279.732874
3,AA4,BSE,Materials,828.13,38182.08,73.66,151.76,8316793.0,4.80,0.03,0.0431,74.9,1.11,5.60,1121.787632
4,AA5,S&P 500,Real Estate,NaN,3351.28,23.79,NaN,3943312.0,8.06,-0.43,0.0244,1.0,1.07,9.46,382.962534


In [19]:
s = df["Stock Index"]

In [20]:
df.drop(["Stock Index"], axis = 1, inplace = True)

In [21]:
df.head()

,Index,Industry,VWAP,General Index,NAV,P/E Ratio,Volumes Traded,Inventory Turnover,Covid Impact (Beta),Tracking Error,Dollar Exchange Rate,Put-Call Ratio,P/B Ratio,Stock Price
0,NYSE,Real Estate,700.29,12765.84,32.38,45.35,1208599.0,5.05,-0.43,0.0520,1.0,0.61,9.20,156.987852
1,NYSE,Information Tech,214.52,12765.84,33.25,170.56,13753356.0,3.10,0.23,0.0164,1.0,NaN,6.91,560.859622
2,NYSE,Information Tech,943.42,12765.84,88.27,143.58,6892003.0,7.70,0.23,0.0396,1.0,NaN,6.09,1279.732874
3,BSE,Materials,828.13,38182.08,73.66,151.76,8316793.0,4.80,0.03,0.0431,74.9,1.11,5.60,1121.787632
4,S&P 500,Real Estate,NaN,3351.28,23.79,NaN,3943312.0,8.06,-0.43,0.0244,1.0,1.07,9.46,382.962534


### Filling the missing values

In [22]:
df_null = df.columns[df.isnull().any()]
df[df_null].isnull().sum()

VWAP                     38
General Index            62
NAV                      61
P/E Ratio               234
Volumes Traded          268
Inventory Turnover      399
Covid Impact (Beta)     376
Tracking Error           71
Dollar Exchange Rate     77
Put-Call Ratio           85
P/B Ratio                25
dtype: int64

In [23]:
df["VWAP"].fillna(method ='ffill', inplace = True)
df["General Index"].fillna(method ='ffill', inplace = True)
df["NAV"].fillna(method ='ffill', inplace = True)
df["P/E Ratio"].fillna(method ='ffill', inplace = True)
df["Volumes Traded"].fillna(method ='ffill', inplace = True)
df["Inventory Turnover"].fillna(method ='ffill', inplace = True)
df["Covid Impact (Beta)"].fillna(method ='ffill', inplace = True)
df["Tracking Error"].fillna(method ='ffill', inplace = True)
df["Dollar Exchange Rate"].fillna(method ='ffill', inplace = True)
df["Put-Call Ratio"].fillna(method ='ffill', inplace = True)
df["P/B Ratio"].fillna(method ='ffill', inplace = True)

In [24]:
df.head()

,Index,Industry,VWAP,General Index,NAV,P/E Ratio,Volumes Traded,Inventory Turnover,Covid Impact (Beta),Tracking Error,Dollar Exchange Rate,Put-Call Ratio,P/B Ratio,Stock Price
0,NYSE,Real Estate,700.29,12765.84,32.38,45.35,1208599.0,5.05,-0.43,0.0520,1.0,0.61,9.20,156.987852
1,NYSE,Information Tech,214.52,12765.84,33.25,170.56,13753356.0,3.10,0.23,0.0164,1.0,0.61,6.91,560.859622
2,NYSE,Information Tech,943.42,12765.84,88.27,143.58,6892003.0,7.70,0.23,0.0396,1.0,0.61,6.09,1279.732874
3,BSE,Materials,828.13,38182.08,73.66,151.76,8316793.0,4.80,0.03,0.0431,74.9,1.11,5.60,1121.787632
4,S&P 500,Real Estate,828.13,3351.28,23.79,151.76,3943312.0,8.06,-0.43,0.0244,1.0,1.07,9.46,382.962534


### Using LabelEncoder to convert Categorical Values into Numerical

In [25]:
df_ctgcol = list(df.select_dtypes(exclude = ["number"]).columns)
df_ctgcol

['Index', 'Industry']

In [26]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in df_ctgcol:
    print(df[i].unique())
    df[i] = le.fit_transform(df[i])
    print(df[i].unique())

['NYSE' 'BSE' 'S&P 500' 'NSE' 'JSE']
[3 0 4 2 1]
['Real Estate' 'Information Tech' 'Materials' 'Healthcare' 'Energy']
[4 2 3 1 0]


In [27]:
df.head()

,Index,Industry,VWAP,General Index,NAV,P/E Ratio,Volumes Traded,Inventory Turnover,Covid Impact (Beta),Tracking Error,Dollar Exchange Rate,Put-Call Ratio,P/B Ratio,Stock Price
0,3,4,700.29,12765.84,32.38,45.35,1208599.0,5.05,-0.43,0.0520,1.0,0.61,9.20,156.987852
1,3,2,214.52,12765.84,33.25,170.56,13753356.0,3.10,0.23,0.0164,1.0,0.61,6.91,560.859622
2,3,2,943.42,12765.84,88.27,143.58,6892003.0,7.70,0.23,0.0396,1.0,0.61,6.09,1279.732874
3,0,3,828.13,38182.08,73.66,151.76,8316793.0,4.80,0.03,0.0431,74.9,1.11,5.60,1121.787632
4,4,4,828.13,3351.28,23.79,151.76,3943312.0,8.06,-0.43,0.0244,1.0,1.07,9.46,382.962534


## Test Dataset - Preprocessing

In [28]:
df1.head()

,Stock Index,Index,Industry,VWAP,General Index,NAV,P/E Ratio,Volumes Traded,Inventory Turnover,Covid Impact (Beta),Tracking Error,Dollar Exchange Rate,Put-Call Ratio,P/B Ratio
0,AC3235,NSE,Materials,53.30,11270.15,44.59,185.09,12067855.0,2.78,0.03,0.0330,74.9,0.80,4.66
1,AC3236,JSE,Energy,749.34,55722.00,74.20,34.01,6172474.0,6.78,0.11,0.0464,17.7,0.86,6.11
2,AC3237,S&P 500,Information Tech,567.75,3351.28,88.41,177.40,17472488.0,4.36,0.23,0.0401,1.0,0.93,4.99
3,AC3238,NSE,Healthcare,646.78,11270.15,79.36,105.44,15553159.0,2.77,0.78,0.0187,74.9,1.09,1.26
4,AC3239,NYSE,Materials,380.33,12765.84,19.90,139.40,12525784.0,5.93,0.03,0.0261,1.0,NaN,6.18


In [29]:
df1.shape

(3331, 14)

### Dropping of unwanted features/attributes

In [30]:
a = df1['Stock Index']

In [31]:
df1.drop(["Stock Index"], axis = 1, inplace = True)

In [32]:
df1.head()

,Index,Industry,VWAP,General Index,NAV,P/E Ratio,Volumes Traded,Inventory Turnover,Covid Impact (Beta),Tracking Error,Dollar Exchange Rate,Put-Call Ratio,P/B Ratio
0,NSE,Materials,53.30,11270.15,44.59,185.09,12067855.0,2.78,0.03,0.0330,74.9,0.80,4.66
1,JSE,Energy,749.34,55722.00,74.20,34.01,6172474.0,6.78,0.11,0.0464,17.7,0.86,6.11
2,S&P 500,Information Tech,567.75,3351.28,88.41,177.40,17472488.0,4.36,0.23,0.0401,1.0,0.93,4.99
3,NSE,Healthcare,646.78,11270.15,79.36,105.44,15553159.0,2.77,0.78,0.0187,74.9,1.09,1.26
4,NYSE,Materials,380.33,12765.84,19.90,139.40,12525784.0,5.93,0.03,0.0261,1.0,NaN,6.18


### Filling the missing values

In [33]:
df1_null = df1.columns[df1.isnull().any()]
df1[df1_null].isnull().sum()

VWAP                     12
General Index           271
NAV                     275
P/E Ratio               292
Volumes Traded           48
Inventory Turnover       30
Covid Impact (Beta)      31
Tracking Error          126
Dollar Exchange Rate    143
Put-Call Ratio          429
P/B Ratio               138
dtype: int64

In [34]:
df1["VWAP"].fillna(method ='ffill', inplace = True)
df1["General Index"].fillna(method ='ffill', inplace = True)
df1["NAV"].fillna(method ='ffill', inplace = True)
df1["P/E Ratio"].fillna(method ='ffill', inplace = True)
df1["Volumes Traded"].fillna(method ='ffill', inplace = True)
df1["Inventory Turnover"].fillna(method ='ffill', inplace = True)
df1["Covid Impact (Beta)"].fillna(method ='ffill', inplace = True)
df1["Tracking Error"].fillna(method ='ffill', inplace = True)
df1["Dollar Exchange Rate"].fillna(method ='ffill', inplace = True)
df1["Put-Call Ratio"].fillna(method ='ffill', inplace = True)
df1["P/B Ratio"].fillna(method ='ffill', inplace = True)

In [35]:
df1.head()

,Index,Industry,VWAP,General Index,NAV,P/E Ratio,Volumes Traded,Inventory Turnover,Covid Impact (Beta),Tracking Error,Dollar Exchange Rate,Put-Call Ratio,P/B Ratio
0,NSE,Materials,53.30,11270.15,44.59,185.09,12067855.0,2.78,0.03,0.0330,74.9,0.80,4.66
1,JSE,Energy,749.34,55722.00,74.20,34.01,6172474.0,6.78,0.11,0.0464,17.7,0.86,6.11
2,S&P 500,Information Tech,567.75,3351.28,88.41,177.40,17472488.0,4.36,0.23,0.0401,1.0,0.93,4.99
3,NSE,Healthcare,646.78,11270.15,79.36,105.44,15553159.0,2.77,0.78,0.0187,74.9,1.09,1.26
4,NYSE,Materials,380.33,12765.84,19.90,139.40,12525784.0,5.93,0.03,0.0261,1.0,1.09,6.18


### Using LabelEncoder to convert Categorical Values into Numerical

In [36]:
df1_ctgcol = list(df1.select_dtypes(exclude = ["number"]).columns)
df1_ctgcol

['Index', 'Industry']

In [37]:
le = LabelEncoder()
for i in df1_ctgcol:
    print(df1[i].unique())
    df1[i] = le.fit_transform(df1[i])
    print(df1[i].unique())

['NSE' 'JSE' 'S&P 500' 'NYSE' 'BSE']
[2 1 4 3 0]
['Materials' 'Energy' 'Information Tech' 'Healthcare' 'Real Estate']
[3 0 2 1 4]


In [38]:
df1.head()

,Index,Industry,VWAP,General Index,NAV,P/E Ratio,Volumes Traded,Inventory Turnover,Covid Impact (Beta),Tracking Error,Dollar Exchange Rate,Put-Call Ratio,P/B Ratio
0,2,3,53.30,11270.15,44.59,185.09,12067855.0,2.78,0.03,0.0330,74.9,0.80,4.66
1,1,0,749.34,55722.00,74.20,34.01,6172474.0,6.78,0.11,0.0464,17.7,0.86,6.11
2,4,2,567.75,3351.28,88.41,177.40,17472488.0,4.36,0.23,0.0401,1.0,0.93,4.99
3,2,1,646.78,11270.15,79.36,105.44,15553159.0,2.77,0.78,0.0187,74.9,1.09,1.26
4,3,3,380.33,12765.84,19.90,139.40,12525784.0,5.93,0.03,0.0261,1.0,1.09,6.18


In [39]:
df.shape, df1.shape

((7383, 14), (3331, 13))

In [40]:
for i in range(len(vals)):
    print(i)
    df['Put-Call Ratio'][i] = vals[i][0]

0


<ipython-input-40-c5544f203b80>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Put-Call Ratio'][i] = vals[i][0]


In [41]:
df.head()

,Index,Industry,VWAP,General Index,NAV,P/E Ratio,Volumes Traded,Inventory Turnover,Covid Impact (Beta),Tracking Error,Dollar Exchange Rate,Put-Call Ratio,P/B Ratio,Stock Price
0,3,4,700.29,12765.84,32.38,45.35,1208599.0,5.05,-0.43,0.0520,1.0,0.895728,9.20,156.987852
1,3,2,214.52,12765.84,33.25,170.56,13753356.0,3.10,0.23,0.0164,1.0,0.610000,6.91,560.859622
2,3,2,943.42,12765.84,88.27,143.58,6892003.0,7.70,0.23,0.0396,1.0,0.610000,6.09,1279.732874
3,0,3,828.13,38182.08,73.66,151.76,8316793.0,4.80,0.03,0.0431,74.9,1.110000,5.60,1121.787632
4,4,4,828.13,3351.28,23.79,151.76,3943312.0,8.06,-0.43,0.0244,1.0,1.070000,9.46,382.962534


In [42]:
df_null = df.columns[df.isnull().any()]
df[df_null].isnull().sum()

Series([], dtype: float64)

In [43]:
df1.head()

,Index,Industry,VWAP,General Index,NAV,P/E Ratio,Volumes Traded,Inventory Turnover,Covid Impact (Beta),Tracking Error,Dollar Exchange Rate,Put-Call Ratio,P/B Ratio
0,2,3,53.30,11270.15,44.59,185.09,12067855.0,2.78,0.03,0.0330,74.9,0.80,4.66
1,1,0,749.34,55722.00,74.20,34.01,6172474.0,6.78,0.11,0.0464,17.7,0.86,6.11
2,4,2,567.75,3351.28,88.41,177.40,17472488.0,4.36,0.23,0.0401,1.0,0.93,4.99
3,2,1,646.78,11270.15,79.36,105.44,15553159.0,2.77,0.78,0.0187,74.9,1.09,1.26
4,3,3,380.33,12765.84,19.90,139.40,12525784.0,5.93,0.03,0.0261,1.0,1.09,6.18


In [44]:
df1_null = df1.columns[df1.isnull().any()]
df1[df1_null].isnull().sum()

Series([], dtype: float64)

### Normalizing the data using MinMaxScaler function

In [45]:
X = df.iloc[:,0:13]
y = df.iloc[:,[13]]

In [46]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
sc2 = MinMaxScaler()
X_train = sc.fit_transform(X)
y = np.reshape(np.array(y), (-1,1))
y_train = sc2.fit_transform(y)

In [47]:
X_train.shape, y_train.shape

((7383, 13), (7383, 1))

In [48]:
U = df1.iloc[:,0:13]

In [49]:
sc = MinMaxScaler()
X_test = sc.fit_transform(U)

In [50]:
X_test.shape

(3331, 13)

### Training of Train Data & Prediction of Test data using the best model discoverd previously

In [51]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [52]:
reg_ann = Sequential()
reg_ann.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu', input_dim = 13))
reg_ann.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
reg_ann.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
reg_ann.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [53]:
reg_ann.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

In [54]:
reg_ann.fit(X_train, y_train, batch_size = 13, epochs = 65)

Epoch 1/65
568/568 [==============================] - 1s 1ms/step - loss: 0.0492 - accuracy: 1.3545e-04
Epoch 2/65
568/568 [==============================] - 1s 2ms/step - loss: 0.0138 - accuracy: 1.3545e-04
Epoch 3/65
568/568 [==============================] - 1s 1ms/step - loss: 0.0087 - accuracy: 2.7089e-04
Epoch 4/65
568/568 [==============================] - 1s 1ms/step - loss: 0.0070 - accuracy: 2.7089e-04
Epoch 5/65
568/568 [==============================] - 1s 1ms/step - loss: 0.0061 - accuracy: 2.7089e-04
Epoch 6/65
568/568 [==============================] - 1s 983us/step - loss: 0.0055 - accuracy: 2.7089e-04
Epoch 7/65
568/568 [==============================] - 1s 999us/step - loss: 0.0051 - accuracy: 2.7089e-04
Epoch 8/65
568/568 [==============================] - 1s 994us/step - loss: 0.0048 - accuracy: 2.7089e-04
Epoch 9/65
568/568 [==============================] - 1s 1ms/step - loss: 0.0046 - accuracy: 2.7089e-04
Epoch 10/65
568/568 [==============================] - 1s 

In [55]:
y_pred = reg_ann.predict(X_test)

In [56]:
y_val = y_pred.reshape(-1,1)
y_val = sc2.inverse_transform(y_val)

In [57]:
y = pd.DataFrame(y_val)
y

,0
0,866.119019
1,298.194427
2,1466.656494
3,794.142578
4,294.959229
...,...
3326,181.447479
3327,183.695984
3328,187.358887
3329,474.733948


### Exporting values into an Excel workbook

In [58]:
test_index = df2.iloc[:,0].values

In [59]:
import xlsxwriter

In [60]:
workbook = xlsxwriter.Workbook('part-02.xlsx') 
worksheet = workbook.add_worksheet()

In [61]:
row = 1
row_id=0
column_id = 0
column=1
worksheet.write(0,0,'Stock Index')
worksheet.write(0,1,'Stock Price (Predicted)')

0

In [62]:
for i in range(len(y_val)) : 
    worksheet.write(row, column_id, test_index[i])
    worksheet.write(row, column, y_val[i])
    row += 1

In [63]:
workbook.close()